# Linear Regression That Actually Works - Features, Interactions, Diagnostics

<hr>

<center>
<div>
<img src="https://raw.githubusercontent.com/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/main/notebooks/figures/mgmt_474_ai_logo_02-modified.png" width="200"/>
</div>
</center>

# <center><a class="tocSkip"></center>
# <center>MGMT47400 Predictive Analytics</center>
# <center>Professor: Davi Moreira </center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/04_linear_features_diagnostics_student.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Fit and interpret linear regression in a pipeline
2. Create interaction/polynomial features responsibly
3. Diagnose underfit/overfit using validation results
4. Use residual analysis to spot nonlinearity and heteroskedasticity
5. Translate coefficients into business meaning (with caveats)

---

> **📋 Participation Reminder:** This notebook contains **2 PAUSE-AND-DO exercises**. You are expected to complete all exercises before submitting your notebook.

---

## 💼 Why This Matters: What Drives the Price?

Your baseline model at **HomeValue Analytics** beats the average, but the CEO wants more: *"Can you tell me which features drive prices the most? Why are coastal houses more expensive?"* Stakeholders don't just want predictions — they want explanations.

Linear regression provides both: a prediction and a coefficient for every feature. But you also need to diagnose where the model struggles — are residuals random, or do they reveal systematic blind spots?

> **Today's focus:** Building an interpretable linear regression for house prices, reading coefficients as business insights, and diagnosing model weaknesses.

---

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
RANDOM_SEED = 474
np.random.seed(RANDOM_SEED)
print("✓ Setup complete!")

**Reading the output:**

The `Setup complete!` message confirms that all imports loaded without error. This notebook's import list extends the previous one with `PolynomialFeatures` (for automatic feature expansion) from scikit-learn's preprocessing module. The same display settings (`display.precision = 4`, `whitegrid` style, `figure.figsize = (10, 6)`) and **RANDOM_SEED = 474** ensure visual and numerical consistency with earlier notebooks.

**Why this matters:** Verifying imports at the top of the notebook catches environment mismatches early. If you are running in Google Colab, all these packages come pre-installed; on a local machine you may need `pip install scikit-learn`.

---

## 1. Load Data and Create Splits

We continue with the California Housing dataset used in the previous notebook. The same 60/20/20 split and `RANDOM_SEED = 474` guarantee that our partitions are identical, making all comparisons across notebooks valid.
Having a consistent baseline dataset lets us isolate the effect of new modeling techniques rather than confounding them with different data.

> 💡 **Gemini Prompt:** "Load the California Housing dataset from scikit-learn as a DataFrame. Separate the target column MedHouseVal from the features. Then split the data into train (60%), validation (20%), and test (20%) sets using a two-step train_test_split with random_state=474. Print the size of each partition."
>
> **After running, verify:**
> - Train set has approximately 12,384 samples (60% of ~20,640)
> - Validation and test sets each have approximately 4,128 samples (20% each)
> - Target variable y contains only MedHouseVal values, not included in X


In [ ]:
california = fetch_california_housing(as_frame=True)
df = california.frame
X = df.drop(columns=['MedHouseVal'])
y = df['MedHouseVal']

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, random_state=RANDOM_SEED)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=RANDOM_SEED)

print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)} (locked)")

**Reading the output:**

The printed summary shows the same partition sizes as the previous notebook: roughly **12,384 training**, **4,128 validation**, and **4,128 test** samples, with the test set marked `(locked)`. Because we use the identical `RANDOM_SEED = 474` and the same two-stage `train_test_split` procedure, every row lands in exactly the same partition as before. This reproducibility is essential: if the splits differed, any performance comparison with the previous notebook would be meaningless.

**Key takeaway:** Consistent splits across notebooks let you attribute performance changes entirely to modeling choices, not to random variation in the data partition.

---

## 2. Baseline Linear Model with Preprocessing

Before adding any feature engineering, we fit the same `StandardScaler` + `LinearRegression` pipeline from the previous notebook. This gives us a clean reference point: the best a plain linear model can do with the original 8 features.
We report both train and validation R² as well as the gap between them, which is our first diagnostic for overfitting.

> 💡 **Gemini Prompt:** "Create a scikit-learn Pipeline with two steps: StandardScaler for feature normalization and LinearRegression as the estimator. Fit it on the training data, then compute and print the R-squared score on both the training and validation sets, along with the overfitting gap (train R-squared minus val R-squared)."
>
> **After running, verify:**
> - Both Train R-squared and Val R-squared are in the range 0.59--0.62
> - The overfit gap is very small (close to 0.00), indicating low variance
> - Pipeline has exactly two named steps: scaler and regressor


In [ ]:
# Simple linear regression pipeline
baseline_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

baseline_pipeline.fit(X_train, y_train)

train_score = baseline_pipeline.score(X_train, y_train)
val_score = baseline_pipeline.score(X_val, y_val)

print("=== BASELINE LINEAR MODEL ===")
print(f"Train R²: {train_score:.4f}")
print(f"Val R²: {val_score:.4f}")
print(f"Overfit gap: {train_score - val_score:.4f}")

**Reading the output:**

The baseline linear model reports **Train R²** and **Val R²**, both likely in the range of **0.60--0.61**. The "Overfit gap" (train minus validation R²) should be very small (under **0.01**), confirming that a plain 8-feature linear model does not overfit on this dataset. This is expected: ordinary linear regression with few features relative to thousands of training samples has low variance.

**Why this matters:** This baseline R² becomes the number to beat. Every feature-engineering step below must be judged by whether it raises validation R² without dramatically increasing the overfit gap.

---

## 3. Coefficient Interpretation

### Understanding Linear Regression Coefficients

**What coefficients tell you:**
- Direction of relationship (positive/negative)
- Relative importance (after scaling)
- Magnitude of effect (with caveats)

**Interpretation caveats:**
- Only valid if features are scaled consistently
- Assumes linear relationship
- Affected by multicollinearity
- Correlation ≠ causation

> 💡 **Gemini Prompt:** "Extract the linear regression coefficients from the fitted baseline pipeline, pair each with its feature name, and sort by absolute value. Display the sorted table, then create a horizontal bar chart showing each coefficient. Also print the intercept and a note explaining that positive coefficients increase predictions while negative ones decrease them."
>
> **After running, verify:**
> - MedInc (median income) has the largest absolute coefficient
> - Bar chart has a red dashed vertical line at x=0 separating positive from negative effects
> - Intercept value is printed below the chart (should be close to the mean of y_train)


In [ ]:
# Extract coefficients
coefficients = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': baseline_pipeline.named_steps['regressor'].coef_,
    'Abs_Coefficient': np.abs(baseline_pipeline.named_steps['regressor'].coef_)
}).sort_values('Abs_Coefficient', ascending=False)

print("=== LINEAR REGRESSION COEFFICIENTS ===")
print(coefficients)

# Visualize
plt.figure(figsize=(10, 6))
plt.barh(coefficients['Feature'], coefficients['Coefficient'])
plt.xlabel('Coefficient Value (after scaling)')
plt.title('Feature Importance via Linear Regression Coefficients')
plt.axvline(x=0, color='red', linestyle='--')
plt.tight_layout()
plt.show()

print(f"\nIntercept: {baseline_pipeline.named_steps['regressor'].intercept_:.4f}")
print(f"\n💡 Positive coefficients increase the prediction")
print(f"💡 Negative coefficients decrease the prediction")

**Reading the output:**

The table lists all 8 features sorted by absolute coefficient magnitude. Because features were standardized first, the coefficient sizes are directly comparable. **MedInc** (median income) almost certainly dominates, reflecting the well-known strong correlation between neighborhood income and house prices. The horizontal bar chart makes this hierarchy visual: long bars indicate influential features, and the direction (positive or negative) shows whether the feature increases or decreases predicted value. The printed intercept is the model's predicted value when all standardized features equal zero (i.e., at the training-set mean of every feature).

**Key takeaway:** Standardized coefficients are useful for ranking feature importance, but remember that they measure *association*, not *causation*. A large coefficient on `Latitude` does not mean moving north causes price changes; it reflects geographic price patterns in the data.

---

## 4. Feature Engineering: Interactions

### When to Add Interactions

Interactions capture **combined effects** of features:
- Income × Location might matter more than either alone
- Age × Size might have nonlinear effects

**Warning:** Interactions increase features exponentially!

> 💡 **Gemini Prompt:** "Manually create an interaction feature by multiplying MedInc and AveRooms columns in both the training and validation sets. Fit a new StandardScaler + LinearRegression pipeline on the augmented training data (now 9 features), evaluate on both sets, and print the R-squared improvement over the baseline model."
>
> **After running, verify:**
> - New column MedInc_x_AveRooms appears in both training and validation DataFrames
> - Feature count increased from 8 to 9
> - Improvement over baseline is printed (may be positive or near-zero)


In [ ]:
# Let's create a simple interaction manually first
# Example: interaction between MedInc and AveRooms
X_train_interact = X_train.copy()
X_val_interact = X_val.copy()

X_train_interact['MedInc_x_AveRooms'] = X_train['MedInc'] * X_train['AveRooms']
X_val_interact['MedInc_x_AveRooms'] = X_val['MedInc'] * X_val['AveRooms']

# Fit model with interaction
interact_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

interact_pipeline.fit(X_train_interact, y_train)

train_score_interact = interact_pipeline.score(X_train_interact, y_train)
val_score_interact = interact_pipeline.score(X_val_interact, y_val)

print("=== WITH INTERACTION FEATURE ===")
print(f"Train R²: {train_score_interact:.4f}")
print(f"Val R²: {val_score_interact:.4f}")
print(f"\nImprovement over baseline: {val_score_interact - val_score:.4f}")

**Reading the output:**

After adding the hand-crafted `MedInc_x_AveRooms` interaction feature, the model now has **9 features** instead of 8. The printed train and validation R² values let you assess whether this single interaction term improved prediction. The "Improvement over baseline" line shows the difference in validation R² -- typically a small positive number (a few thousandths) for a single interaction. This modest gain reflects the fact that income-times-rooms captures a joint effect not fully represented by either variable alone.

**Why this matters:** Manually crafting interactions forces you to think about domain knowledge: *which* feature combinations have a plausible combined effect? This is more interpretable than blindly generating all possible interactions.

---

## 📝 PAUSE-AND-DO Exercise 1 (5 minutes)

**Task:** Add an interaction or polynomial block and measure validation change.

**Instructions:**
1. Use `PolynomialFeatures` to create degree=2 features
2. Fit a new pipeline
3. Compare validation scores
4. Watch for overfitting!

---

> 💡 **Gemini Prompt:** "Build a three-step Pipeline that first applies PolynomialFeatures with degree=2 and include_bias=False, then StandardScaler, then LinearRegression. Fit on the original training data, score on both train and validation, and print the R-squared values, overfit gap, original feature count, expanded feature count, and the feature-explosion multiplier."
>
> **After running, verify:**
> - Feature count expands from 8 to 44 (originals + squares + pairwise products)
> - Train R-squared is noticeably higher than baseline (likely above 0.65)
> - Overfit gap is larger than baseline, signaling increased model complexity


In [ ]:
# YOUR CODE HERE: Add polynomial features

poly_pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

poly_pipeline.fit(X_train, y_train)

train_score_poly = poly_pipeline.score(X_train, y_train)
val_score_poly = poly_pipeline.score(X_val, y_val)

print("=== POLYNOMIAL FEATURES (degree=2) ===")
print(f"Train R²: {train_score_poly:.4f}")
print(f"Val R²: {val_score_poly:.4f}")
print(f"Overfit gap: {train_score_poly - val_score_poly:.4f}")

# Count features
n_features_original = X_train.shape[1]
n_features_poly = poly_pipeline.named_steps['poly'].transform(X_train[:1]).shape[1]
print(f"\nOriginal features: {n_features_original}")
print(f"After polynomial (degree=2): {n_features_poly}")
print(f"Feature explosion: {n_features_poly / n_features_original:.1f}x increase")

**Reading the output:**

With `PolynomialFeatures(degree=2)`, the feature count explodes from **8 to 44** (all original features plus their squares and pairwise products). The printed R² values show a noticeable jump in both train and validation scores compared to the baseline. However, the overfit gap (train R² minus validation R²) is now larger, signaling that the added complexity is starting to memorize training-set noise. The "Feature explosion" line quantifies the cost: a **5.5x increase** in feature dimensionality.

**Key takeaway:** Polynomial features offer a quick accuracy boost, but the curse of dimensionality is real. With 44 features and only ~12,000 training samples, you are approaching the regime where regularization (covered in the next notebook) becomes essential.

---

### YOUR ANALYSIS:

**Observation 1: Performance**  
[Did polynomial features improve validation score?]

**Observation 2: Overfitting**  
[Is the train-val gap larger now?]

**Observation 3: Complexity**  
[Is the added complexity worth the improvement?]

---

## 5. Residual Diagnostics

### What to Look For:

1. **Patterns in residual plots** → Model is missing something (nonlinearity, interactions)
2. **Funnel shape** → Heteroskedasticity (variance changes with prediction)
3. **Non-normal residuals** → Outliers or wrong model family
4. **Systematic errors in ranges** → Model struggles in certain regions

> 💡 **Gemini Prompt:** "Generate validation predictions from both the baseline and polynomial pipelines. Compute residuals for each. Create a 2x2 subplot grid: top row shows scatter plots of residuals vs. predicted values (baseline left, polynomial right, each with a red dashed y=0 line), bottom row shows histograms of residual distributions (50 bins). Print MAE and RMSE for both models."
>
> **After running, verify:**
> - Top-row scatter plots are centered around the y=0 reference line
> - Polynomial model residuals (orange) show a tighter spread than baseline (blue)
> - Printed MAE and RMSE for the polynomial model are lower than for the baseline


In [ ]:
# Get predictions
y_pred_baseline = baseline_pipeline.predict(X_val)
y_pred_poly = poly_pipeline.predict(X_val)

residuals_baseline = y_val - y_pred_baseline
residuals_poly = y_val - y_pred_poly

# Residual comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Baseline residuals
axes[0, 0].scatter(y_pred_baseline, residuals_baseline, alpha=0.5)
axes[0, 0].axhline(y=0, color='r', linestyle='--')
axes[0, 0].set_xlabel('Predicted')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].set_title('Baseline Model Residuals')

# Polynomial residuals
axes[0, 1].scatter(y_pred_poly, residuals_poly, alpha=0.5, color='orange')
axes[0, 1].axhline(y=0, color='r', linestyle='--')
axes[0, 1].set_xlabel('Predicted')
axes[0, 1].set_ylabel('Residuals')
axes[0, 1].set_title('Polynomial Model Residuals')

# Baseline distribution
axes[1, 0].hist(residuals_baseline, bins=50, edgecolor='black', alpha=0.7)
axes[1, 0].axvline(x=0, color='r', linestyle='--')
axes[1, 0].set_xlabel('Residual')
axes[1, 0].set_title('Baseline Residual Distribution')

# Polynomial distribution
axes[1, 1].hist(residuals_poly, bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1, 1].axvline(x=0, color='r', linestyle='--')
axes[1, 1].set_xlabel('Residual')
axes[1, 1].set_title('Polynomial Residual Distribution')

plt.tight_layout()
plt.show()

print("=== RESIDUAL STATISTICS ===")
print(f"Baseline MAE: {mean_absolute_error(y_val, y_pred_baseline):.4f}")
print(f"Polynomial MAE: {mean_absolute_error(y_val, y_pred_poly):.4f}")
print(f"\nBaseline RMSE: {np.sqrt(mean_squared_error(y_val, y_pred_baseline)):.4f}")
print(f"Polynomial RMSE: {np.sqrt(mean_squared_error(y_val, y_pred_poly)):.4f}")

**Reading the output:**

The 2x2 grid compares residual behavior for the baseline linear model (left column) and the polynomial model (right column). **Top row** (residuals vs. predicted): look for patterns. A funnel shape means the model's error variance changes with the prediction level (heteroskedasticity). A curve means the model misses a nonlinear trend. **Bottom row** (residual histograms): a symmetric, bell-shaped distribution centered at zero is ideal. Heavy tails or skew indicate outliers or systematic mis-specification. The printed MAE and RMSE values quantify the improvement: the polynomial model should show lower error on both metrics.

**Why this matters:** Residual plots reveal *what kind* of error your model makes, not just *how much*. Two models can have similar R² but very different residual patterns. A model with random residuals is trustworthy; a model with patterned residuals is hiding systematic bias that could be fixed with better features or a different model family.

---

## 📝 PAUSE-AND-DO Exercise 2 (5 minutes)

**Task:** Write a short diagnostic conclusion (what error patterns suggest).

Look at the residual plots above and answer:
1. Do you see any patterns (non-random scatter)?
2. Is there a funnel shape (heteroskedasticity)?
3. Does the polynomial model fix any issues?
4. What would you try next?

---

### YOUR DIAGNOSTIC CONCLUSION:

**Pattern Analysis:**  
[Describe any patterns you see in residuals]

**Heteroskedasticity:**  
[Is variance constant across predictions?]

**Model Comparison:**  
[Which model has better residual behavior?]

**Next Steps:**  
[What would you try to improve the model?]

---

## 6. Model Comparison Table

With three models trained (baseline linear, manual interaction, and degree-2 polynomial), we consolidate their performance into a single DataFrame. This side-by-side view makes it easy to compare R², MAE, and the overfit gap in one glance.
The table also records the number of input features for each model, highlighting the cost of polynomial expansion.

> 💡 **Gemini Prompt:** "Create a pandas DataFrame comparing three models (Baseline Linear, With Interaction, Polynomial deg=2) with columns for model name, feature count, Train R-squared, Val R-squared, overfit gap, and validation MAE. Print the table without the index. Then identify and print which model achieved the best validation R-squared."
>
> **After running, verify:**
> - Table has exactly 3 rows and 6 columns (Model, Features, Train_R2, Val_R2, Overfit_Gap, Val_MAE)
> - Feature counts are 8, 9, and 44 respectively
> - Best model line names the model with the highest Val_R2 and prints its score


In [ ]:
# Create comparison table
comparison = pd.DataFrame([
    {
        'Model': 'Baseline Linear',
        'Features': X_train.shape[1],
        'Train_R2': train_score,
        'Val_R2': val_score,
        'Overfit_Gap': train_score - val_score,
        'Val_MAE': mean_absolute_error(y_val, y_pred_baseline)
    },
    {
        'Model': 'With Interaction',
        'Features': X_train_interact.shape[1],
        'Train_R2': train_score_interact,
        'Val_R2': val_score_interact,
        'Overfit_Gap': train_score_interact - val_score_interact,
        'Val_MAE': mean_absolute_error(y_val, interact_pipeline.predict(X_val_interact))
    },
    {
        'Model': 'Polynomial (deg=2)',
        'Features': n_features_poly,
        'Train_R2': train_score_poly,
        'Val_R2': val_score_poly,
        'Overfit_Gap': train_score_poly - val_score_poly,
        'Val_MAE': mean_absolute_error(y_val, y_pred_poly)
    }
])

print("=== MODEL COMPARISON ===")
print(comparison.to_string(index=False))

# Best model
best_idx = comparison['Val_R2'].idxmax()
print(f"\n✓ Best validation R²: {comparison.loc[best_idx, 'Model']} ({comparison.loc[best_idx, 'Val_R2']:.4f})")

**Reading the output:**

The final comparison table shows three rows: **Baseline Linear** (8 features), **With Interaction** (9 features), and **Polynomial deg=2** (44 features). The columns include train R², validation R², the overfit gap, and validation MAE. The polynomial model likely wins on validation R², but its overfit gap is the largest. The "Best validation R²" line at the bottom names the winner. Notice the cost-benefit tradeoff: going from 8 to 44 features may yield only a few percentage points of R² improvement while substantially increasing model complexity and the risk of overfitting.

**Key takeaway:** Always weigh accuracy gains against complexity costs. A simpler model that performs almost as well is often preferable in production because it is easier to explain, faster to train, and less likely to degrade on new data.

---

## 7. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Coefficient Interpretation**: Requires scaling, caution about multicollinearity
2. **Feature Engineering**: Interactions and polynomials can help, but watch overfitting
3. **Residual Diagnostics**: Visual patterns reveal model inadequacies
4. **Complexity Tradeoff**: More features ≠ better performance
5. **Validation Discipline**: Always check validation scores, not just training

### Critical Rules:

> **"Scale features before interpreting coefficients"**

> **"Polynomial features explode exponentially - use sparingly"**

> **"Residual plots never lie"**

### Next Steps:

- Next notebook: Regularization (Ridge/Lasso) + Project proposal
- Regularization will help control overfitting from complex features
- Start thinking about your project dataset and target

---

## Participation Assignment Submission Instructions

### To Submit This Notebook:

1. **Complete all exercises**: Fill in both PAUSE-AND-DO exercise cells with your findings
2. **Run All Cells**: Execute `Runtime → Run all` to ensure everything works
3. **Save a Copy**: `File → Save a copy in Drive or Download the .ipynb extension`
4. **Submit**: Upload your `.ipynb` file in the participation assignment you find in the course Brightspace page.

### Before Submitting, Check:

- [ ] All cells execute without errors
- [ ] All outputs are visible
- [ ] Both exercise responses are complete
- [ ] Notebook is shared with correct permissions
- [ ] You can explain every line of code you wrote

### Next Step:

Complete the **Quiz** in Brightspace (auto-graded)

---

## Bibliography

- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). *An Introduction to Statistical Learning with Python* - Linear Regression chapter
- Hastie, T., Tibshirani, R., & Friedman, J. (2009). *The Elements of Statistical Learning* - Linear methods
- scikit-learn User Guide: [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
- scikit-learn User Guide: [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

---



<center>

Thank you!

</center>